# Data Understanding

Nesta etapa o objetivo é entender profundamente os dados disponíveis,
avaliando sua estrutura, qualidade, volume e possíveis limitações.

Não há, neste momento, qualquer tipo de tratamento ou limpeza dos dados.
O foco é responder perguntas como:
- Quais dados estão disponíveis?
- Como eles estão estruturados?
- Existem valores ausentes?
- Há dados sensíveis?
- O que pode impactar as análises futuras?

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# Definindo quantas colunas o pandas irá exibir
pd.set_option("display.max_columns", 100)

# Definindo como números float serão exibidos (2 casas decimais)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

- Lendo os arquivos CSV

In [3]:
DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

len(csv_files), csv_files

(12,
 [WindowsPath('../data/raw/abr2025.csv'),
  WindowsPath('../data/raw/ago2025.csv'),
  WindowsPath('../data/raw/dez2025.csv'),
  WindowsPath('../data/raw/fev2025.csv'),
  WindowsPath('../data/raw/jan2025.csv'),
  WindowsPath('../data/raw/jul2025.csv'),
  WindowsPath('../data/raw/jun2025.csv'),
  WindowsPath('../data/raw/mai2025.csv'),
  WindowsPath('../data/raw/mar2025.csv'),
  WindowsPath('../data/raw/nov2025.csv'),
  WindowsPath('../data/raw/out2025.csv'),
  WindowsPath('../data/raw/set2025.csv')])

In [4]:
dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)

df_raw.shape

(24009, 16)

Após importar todos os arquivos ".csv" e concatenar todos em um único dataframe, ficamos com **24009 linhas** e **16 colunas**.

## Observações sobre codificação dos arquivos

Durante a ingestão dos dados, foi identificado que os arquivos CSV
não utilizam codificação UTF-8, resultando em erros de leitura.

Após análise, foi adotada a codificação `latin1`, comum em dados
oriundos de sistemas legados da administração pública brasileira,
permitindo a correta leitura de caracteres acentuados.


In [5]:
df_raw.head()

,Referência,Nome,Cargo,Data Admissão,Tipo de Regime,Descontos,Liquido,Data Desligamento,Proventos,Contrato,Atividade,Nome Atividade,Tipo de Contrato,Data Prevista Termino Contrato,Carga Horária (Sem.),arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,30.00,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.00,NaN,NaN,Efetivo,NaN,30.00,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv


In [6]:
df_raw.tail()

,Referência,Nome,Cargo,Data Admissão,Tipo de Regime,Descontos,Liquido,Data Desligamento,Proventos,Contrato,Atividade,Nome Atividade,Tipo de Contrato,Data Prevista Termino Contrato,Carga Horária (Sem.),arquivo_origem
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,set2025.csv
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.00,NaN,NaN,Efetivo,NaN,40.00,set2025.csv
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.00,NaN,NaN,Efetivo,NaN,30.00,set2025.csv
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,30.00,set2025.csv
24008,NaN,NaN,NaN,NaN,NaN,"918.286,26","4.302.011,08",NaN,"5.220.297,34",NaN,NaN,NaN,NaN,NaN,NaN,set2025.csv


## Dicionário de dados

- **Referência**: Indica o tipo de pagamento realizado e o mês de referência da folha.

- **Nome**: Nome completo do servidor público.

- **Cargo**: Cargo ocupado pelo servidor no período analisado.

- **Data Admissão**: Data de ingresso do servidor no quadro da prefeitura.

- **Tipo de Regime**: Regime de Previdência Social ao qual o servidor está vinculado.

- **Descontos**: Valor total de descontos aplicados diretamente no holerite do servidor.

- **Liquido**: Valor líquido efetivamente recebido pelo servidor após os descontos.

- **Data Desligamento**: Data de desligamento do servidor, quando aplicável.

- **Proventos**: Valor bruto da remuneração do servidor antes da aplicação dos descontos.

- **Contrato**: Código interno utilizado pelo sistema da prefeitura para identificação contratual.

- **Atividade**: Código interno utilizado para classificar funções ou atividades específicas no sistema municipal.

- **Nome Atividade**: Descrição textual da atividade ou função exercida pelo servidor.

- **Tipo de Contrato**: Modalidade de vínculo do servidor com a prefeitura.

- **Data Prevista Termino Contrato**: Data prevista para o encerramento do contrato, quando aplicável.

- **Carga Horária (Sem.)**: Carga horária semanal de trabalho do servidor, expressa em horas.

Observação: como podemos ver na última linha do dataframe, a última linha de cada mês é a soma total das seguintes colunas: Descontos, Liquido e Proventos. 

- Lendo apenas o cabeçalho de vários arquivos CSV e criando um dicionário que mapeia cada arquivo para o conjunto de colunas que ele possui.

In [7]:
schemas = {
    file.name: set(pd.read_csv(file, encoding="latin1", sep=";", nrows=0).columns)
    for file in csv_files
}

schemas

{'abr2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'ago2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'dez2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'fev2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'D

- Identificando quantas estruturas de colunas diferentes existem entre todos os arquivos CSV.

In [8]:
unique_schemas = set(tuple(sorted(cols)) for cols in schemas.values())
len(unique_schemas)

1

Todos os arquivos têm exatamente o mesmo schema.

- Colunas do dataframe

In [9]:
df_raw.columns

Index(['Referência', 'Nome', 'Cargo', 'Data Admissão', 'Tipo de Regime',
       'Descontos', 'Liquido', 'Data Desligamento', 'Proventos', 'Contrato',
       'Atividade', 'Nome Atividade', 'Tipo de Contrato',
       'Data Prevista Termino Contrato', 'Carga Horária (Sem.)',
       'arquivo_origem'],
      dtype='object')

## Análise de valores ausentes (missing values)

A análise abaixo apresenta a proporção de valores ausentes por coluna, calculada como a razão entre o número de valores nulos e o total de registros do dataset.

Essa verificação tem como objetivo avaliar a qualidade inicial dos dados e identificar colunas que demandam atenção especial antes das etapas de preparação e análise.

In [10]:
(df_raw.isna().sum() / len(df_raw)).sort_values(ascending=False)

Data Prevista Termino Contrato   1.00
Nome Atividade                   1.00
Atividade                        1.00
Data Desligamento                0.99
Descontos                        0.50
Proventos                        0.00
Tipo de Contrato                 0.00
Referência                       0.00
Nome                             0.00
Tipo de Regime                   0.00
Cargo                            0.00
Data Admissão                    0.00
Carga Horária (Sem.)             0.00
Contrato                         0.00
Liquido                          0.00
arquivo_origem                   0.00
dtype: float64

### Principais observações:

- As colunas Data Prevista Termino Contrato, Atividade e Nome Atividade apresentam 100% de valores ausentes, indicando que não possuem informação útil para este conjunto de dados.
Essas colunas não serão consideradas nas análises subsequentes.

- A coluna Data Desligamento possui aproximadamente 99% de valores ausentes, o que é consistente com o contexto de servidores públicos, já que a maioria dos vínculos permanece ativa.

- A coluna Descontos apresenta cerca de 50% de valores ausentes, explicados pela natureza do pagamento:

    - Registros referentes a vale-alimentação não possuem descontos associados.

    - Registros de folha mensal apresentam valores preenchidos.

- As demais colunas (Proventos, Liquido, Cargo, Nome, Tipo de Contrato, Data Admissão, entre outras) não apresentam valores ausentes, indicando boa consistência estrutural do dataset.

#### Conclusão:

De forma geral, o conjunto de dados apresenta boa qualidade, com valores ausentes concentrados em colunas que:

- Não são relevantes para as análises propostas, ou
- Possuem ausência explicável pelo contexto administrativo dos dados.

## Tipos de dados

In [11]:
df_raw.dtypes

Referência                         object
Nome                               object
Cargo                              object
Data Admissão                      object
Tipo de Regime                     object
Descontos                          object
Liquido                            object
Data Desligamento                  object
Proventos                          object
Contrato                          float64
Atividade                         float64
Nome Atividade                     object
Tipo de Contrato                   object
Data Prevista Termino Contrato    float64
Carga Horária (Sem.)              float64
arquivo_origem                     object
dtype: object

In [12]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24009 entries, 0 to 24008
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Referência                      23997 non-null  object 
 1   Nome                            23997 non-null  object 
 2   Cargo                           23997 non-null  object 
 3   Data Admissão                   23997 non-null  object 
 4   Tipo de Regime                  23997 non-null  object 
 5   Descontos                       12045 non-null  object 
 6   Liquido                         24009 non-null  object 
 7   Data Desligamento               157 non-null    object 
 8   Proventos                       23931 non-null  object 
 9   Contrato                        23997 non-null  float64
 10  Atividade                       24 non-null     float64
 11  Nome Atividade                  24 non-null     object 
 12  Tipo de Contrato                

### Estrutura do dataset e tipos de dados

A inspeção dos tipos de dados (`dtypes`) e das informações gerais do DataFrame (`info`) permite avaliar a estrutura inicial do dataset, bem como identificar inconsistências de tipagem e possíveis necessidades de tratamento.

#### Principais pontos observados

- O dataset possui **24.009 registros** e **16 colunas**, ocupando aproximadamente **3 MB de memória**.

- A maioria das colunas está representada como `object`, incluindo:
  - Campos textuais (Nome, Cargo, Tipo de Contrato)
  - Campos numéricos monetários (Proventos, Descontos, Liquido)
  - Campos temporais (Data Admissão, Data Desligamento)

- Colunas originalmente numéricas ou categóricas aparecem como `object` devido a:
  - Uso de formatação brasileira para valores monetários
  - Representação de datas como texto
  - Presença de valores ausentes

- As colunas **Contrato**, **Atividade**, **Data Prevista Termino Contrato** e **Carga Horária (Sem.)** estão tipadas como `float64`, embora algumas representem códigos ou valores inteiros, o que será avaliado durante a etapa de preparação dos dados.

#### Conclusão

O dataset apresenta uma estrutura consistente, porém requer:

- Conversão adequada de valores monetários para `float`
- Padronização de campos de data
- Avaliação do uso e relevância de colunas com baixa taxa de preenchimento

## Estatísticas descritivas das colunas numéricas

A análise das estatísticas descritivas das colunas numéricas permite compreender a distribuição dos valores, identificar colunas com baixa variabilidade e avaliar sua utilidade analítica.

In [13]:
df_raw.describe()

,Contrato,Atividade,Data Prevista Termino Contrato,Carga Horária (Sem.)
count,"23,997.00",24.00,0.00,"23,997.00"
mean,1.05,10.00,NaN,35.34
std,0.22,0.00,NaN,7.48
min,1.00,10.00,NaN,6.00
25%,1.00,10.00,NaN,30.00
50%,1.00,10.00,NaN,40.00
75%,1.00,10.00,NaN,40.00
max,2.00,10.00,NaN,44.00


#### Principais observações

- **Contrato**
  - Possui 23.997 registros válidos.
  - Valores variam entre **1.0 e 2.0**, com média próxima de **1.05**.
  - Baixa variabilidade, indicando que a maior parte dos registros pertence a um mesmo tipo de contrato.
  - Trata-se de um **código interno**, sem significado analítico direto no momento.

- **Atividade**
  - Apenas **24 registros preenchidos** em todo o dataset.
  - Valor constante (**10.0**), sem qualquer variação.
  - Não possui poder informativo ou analítico, reforçando a decisão de não utilizá-la nas análises.

- **Data Prevista Termino Contrato**
  - Não possui registros preenchidos (`count = 0`).
  - A ausência total de dados inviabiliza qualquer análise baseada nessa coluna.
  - Será desconsiderada nas etapas analíticas.

- **Carga Horária (Sem.)**
  - Possui 23.997 registros válidos.
  - Valores variam entre **6 e 44 horas semanais**.
  - Mediana de **40 horas**, indicando que a maioria dos servidores possui jornada padrão.
  - Coluna relevante para análises comparativas entre cargos, categorias e tipos de contrato.

#### Conclusão

A análise estatística confirma que:

- Algumas colunas numéricas representam apenas **códigos internos** ou possuem **baixa ou nenhuma variabilidade**.
- A coluna **Carga Horária (Sem.)** apresenta alto potencial analítico.
- As colunas **Atividade** e **Data Prevista Termino Contrato** não agregam valor para as análises e podem ser descartadas ou ignoradas nas etapas seguintes.

## Inspeção inicial das colunas de data

In [14]:
df_raw["Data Admissão"].head()

0    08/04/1991
1    08/04/1991
2    04/12/2023
3    04/12/2023
4    08/04/1991
Name: Data Admissão, dtype: object

In [15]:
df_raw["Data Desligamento"].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Data Desligamento, dtype: object

#### Observações

- **Data Admissão**
  - Os valores estão representados como texto (`object`).
  - O formato segue o padrão brasileiro `DD/MM/AAAA`.
  - Não há informações de horário, apenas a data de ingresso do servidor.
  - A coluna apresenta alto grau de preenchimento, indicando boa qualidade do dado.

- **Data Desligamento**
  - A maioria dos registros está preenchida com valores nulos (`NaN`).
  - A ausência de data indica, na maior parte dos casos, que o servidor permanece ativo.
  - Quando presente, a coluna representa o desligamento formal do servidor da prefeitura.

#### Implicações para o tratamento dos dados

- Ambas as colunas necessitam de **padronização de formato**, uma vez que estão armazenadas como texto.
- A coluna **Data Desligamento** deve ser tratada de forma especial, considerando que valores nulos possuem significado semântico (servidor ativo).
- Decisões sobre conversão de tipo e criação de variáveis derivadas serão realizadas na etapa de **Data Preparation**.

## Resumo da compreensão dos dados

Nesta etapa de **Data Understanding**, foi realizada uma análise exploratória inicial com foco na estrutura, qualidade e consistência dos dados provenientes do Portal da Transparência.

Principais pontos consolidados:

- Os dados estão distribuídos em múltiplos arquivos CSV, organizados por mês, todos com **schema consistente**, permitindo concatenação segura.
- A codificação utilizada é `latin1`, adequada para lidar com caracteres especiais presentes nos dados originais.
- O dataset consolidado possui **24.009 registros e 16 colunas**, com tamanho aproximado de 3 MB.
- A análise de tipos (`dtypes`) revelou que diversas colunas estão inicialmente tipadas como `object`, incluindo:
  - Valores monetários
  - Datas
  - Campos categóricos
- Foi identificado o uso de **formatação brasileira** para valores numéricos e datas, justificando a tipagem inadequada no estado bruto.
- A análise de valores ausentes mostrou:
  - Colunas com preenchimento quase total (ex.: Nome, Cargo, Proventos).
  - Colunas com alta taxa de valores nulos, como **Data Desligamento**, **Atividade**, **Nome Atividade** e **Data Prevista Término Contrato**, cujos significados foram interpretados de forma contextual.
- As colunas de data estão representadas como texto no formato `DD/MM/AAAA`, sem informação de horário.
- A inspeção estatística das colunas numéricas indicou valores coerentes com a realidade administrativa (ex.: carga horária semanal).
- Foi construído um **dicionário de dados**, documentando o significado de cada coluna e apoiando a compreensão semântica do dataset.

Conclusão:

Os dados apresentam **boa consistência estrutural e semântica**, sendo adequados para análises, desde que passem por tratamentos de tipagem, padronização e seleção de colunas, os quais serão realizados na etapa de **Data Preparation**.

